In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [49]:
data1 = [[ 0.5,  0.7, None, None,  0.3,  0.2,  0.5],
         [ 0.1,  0.2, None, 0.3 , None,  0.6, None],
         [ 0.2, None,  0.8, None, None, None, 0.6 ],
         [0.05, 0.03, None, 0.07,  0.1, None, 0.08],
         [ None, None, 0.01, None, 0.05, None, 0.05]]
data2 = [[500, 1000, 300, 300,  800, 200, 100],
         [600,  500, 200,   0,  400, 300, 150],
         [300,  600,   0,   0,  500, 400, 100],
         [200,  300, 400, 500,  200,   0, 100],
         [  0,  100, 500, 100, 1000, 300,   0],
         [500,  500, 100, 300, 1100, 500,  60]]
data3 = [[3, 2, 3, 1, 1],
         [4, 2, 1, 1, 1],
         [4, 2, 3, 0, 1],
         [4, 1, 3, 1, 1],
         [3, 1, 3, 1, 1],
         [4, 2, 2, 1, 0]]

PRODS = ['PROD1', 'PROD2', 'PROD3', 'PROD4', 'PROD5', 'PROD6', 'PROD7']
MONTHS = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']
MACHINES = ['grinder', 'v_driller', 'h_driller', 'borer', 'planer']
PROCESSES = ['grinding', 'v_drilling', 'h_drilling', 'boring', 'planing']
PREV_MONTH = {'JAN': 'INITIAL', 
              'FEB': 'JAN',
              'MAR': 'FEB',
              'APR': 'MAR',
              'MAY': 'APR',
              'JUN': 'MAY'}

req_proc = pd.DataFrame(data1, columns=PRODS, index=MACHINES)
mark_lim = pd.DataFrame(data2, columns=PRODS, index=MONTHS)
avail_mac = pd.DataFrame(data3, columns=MACHINES, index=MONTHS)
contr_profit = dict(zip(PRODS, [10, 6, 8, 4, 11, 9, 3]))

storage_cost   = 0.5
storage_limit  = 100
shift_length   = 8
n_of_shifts    = 2
days_per_month = 24

In [50]:
model = gp.Model('Factory Planning I')

# add vars
make = model.addVars(PRODS, MONTHS,
                     name='make')
sell = model.addVars(PRODS, MONTHS,
                     name='sell')
store = model.addVars(PRODS, MONTHS+['INITIAL'],
                       name='store')

# set upper and lower bounds of "store" variables
for i in PRODS:
    store[i,'INITIAL'].setAttr('ub', 0.0)
    store[i,'INITIAL'].setAttr('lb', 0.0)
    store[i,'JUN'].setAttr('ub', 50.0)
    store[i,'JUN'].setAttr('lb', 50.0)
    
# objective function
model.setObjective(gp.quicksum(contr_profit[i]*sell[i,t] - storage_cost*store[i,t] for i in PRODS for t in MONTHS),
                    GRB.MAXIMIZE)

# add constraints
model.addConstrs((sell[i,t] <= mark_lim[i][t] for i in PRODS for t in MONTHS),
                  name='market_lim')
model.addConstrs((store[i,t] <= 100 for i in PRODS for t in MONTHS),
                  name='storage_lim')
model.addConstrs((make[i,t] + store[i,PREV_MONTH[t]] - sell[i,t] - store[i,t] == 0 for i in PRODS for t in MONTHS),
                  name='storage_link')
model.addConstrs((gp.quicksum(req_proc[i][m]*make[i,t] for i in PRODS) <= n_of_shifts*shift_length*days_per_month*avail_mac[m][t] for t in MONTHS for m in MACHINES),
                  name='mac_lim')

model.update()

In [51]:
model.write('Factory Planning I.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 156 rows, 133 columns and 372 nonzeros
Model fingerprint: 0x593db255
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [5e+01, 5e+01]
  RHS range        [6e+01, 2e+03]
Presolve removed 151 rows and 117 columns
Presolve time: 0.01s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2466500e+05   3.640000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 2
       4    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds
Optimal objective  9.371517857e+04


In [52]:
print('Profit of ${:.2f}'.format(model.objVal))

sell_solution = pd.DataFrame([], columns=PRODS, index=MONTHS)
sell_solution = sell_solution.fillna(0)
for i, t in sell.keys():
    sell_solution[i][t] = sell[i,t].x
print('===SELLING PLAN')
print(sell_solution)

make_solution = pd.DataFrame([], columns=PRODS, index=MONTHS)
make_solution = make_solution.fillna(0)
for i, t in make.keys():
    make_solution[i][t] = make[i,t].x
print('===PRODUCE PLAN')
print(make_solution)

store_solution = pd.DataFrame([], columns=PRODS, index=['INITIAL']+MONTHS)
store_solution = store_solution.fillna(0)
for i, t in store.keys():
    store_solution[i][t] = store[i,t].x
print('===STORAGE PLAN')
print(store_solution[1:])

Profit of $93715.18
===SELLING PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN    500    888    300    300    800    200      0
FEB    600    500    200      0    400    300    150
MAR    100    100      0      0    100    400    100
APR    200    300    400    500    200      0    100
MAY      0    100    500    100   1000    300      0
JUN    500    500     50    300     50    500     50
===PRODUCE PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN    500    888    382    300    800    200      0
FEB    700    600    117      0    500    300    250
MAR      0      0      0      0      0    400      0
APR    200    300    400    500    200      0    100
MAY      0    100    600    100   1100    300    100
JUN    550    550      0    350      0    550      0
===STORAGE PLAN
     PROD1  PROD2  PROD3  PROD4  PROD5  PROD6  PROD7
JAN      0      0     82      0      0      0      0
FEB    100    100      0      0    100      0    100
MAR      0      0      0      0

In [87]:
print('Shadow Prices of Machines Capacities')

for const in model.getConstrs():
    if const.PI > 0 and const.ConstrName[:7] == 'mac_lim':
        print(const.ConstrName, const.PI)

Shadow Prices of Machines Capacities
mac_lim[JAN,grinder] 8.571428571428571
mac_lim[FEB,h_driller] 0.625
mac_lim[MAR,borer] 200.0
mac_lim[JUN,planer] 800.0


In [80]:
print('Reduced Costs')
for i,t in sell.keys():
    if sell[i,t].RC < 0:
        print(i, t, sell[i,t].RC)

Reduced Costs
PROD7 JAN -1.2857142857142856
